In [1]:
import yfinance as yf
import pandas as pd

In [2]:
data = yf.download("PBW", start="2010-01-01", end="2022-03-19")

[*********************100%***********************]  1 of 1 completed


In [3]:
df = pd.DataFrame(data)

In [4]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,55.450001,56.750000,55.450001,56.750000,43.456406,107960
2010-01-05,56.849998,57.750000,56.450001,57.400002,43.954144,101740
2010-01-06,57.500000,58.549999,57.450001,58.250000,44.605045,181180
2010-01-07,58.250000,58.400002,57.700001,58.099998,44.490166,97420
2010-01-08,58.000000,58.799999,58.000000,58.799999,45.026199,75240


### Relative Strength Index (RSI)

In [5]:
delta = df['Close'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
df['RSI'] = 100 - (100/(1 + rs))

In [6]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,RSI
Date,,,,,,,
2010-01-04,55.450001,56.750000,55.450001,56.750000,43.456406,107960,NaN
2010-01-05,56.849998,57.750000,56.450001,57.400002,43.954144,101740,100.000000
2010-01-06,57.500000,58.549999,57.450001,58.250000,44.605045,181180,100.000000
2010-01-07,58.250000,58.400002,57.700001,58.099998,44.490166,97420,98.292669
2010-01-08,58.000000,58.799999,58.000000,58.799999,45.026199,75240,98.427588
